<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data : https://data.stackexchange.com/stackoverflow/query/edit/1086274
    - Source : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse
    - Entrainement
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 06/08/2019 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Data</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I.1 - Source</p>

Revoir les explications

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I.2 - Preparation des données</p>

In [1]:
import sys
import os
sys.path.insert(0, './Python/')




In [2]:

from tagText import TagText

In [3]:
tagText = TagText()
print(tagText.urlDirectory)
print(tagText.fileName)
data_question = tagText.read_source()
data_question.head()

Data/
QuestionVsTags.csv


,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [4]:
print(tagText.stopwords)
#print(tagText.badwords)

{'had', 'my', "hadn't", "doesn't", 'to', 'wasn', 'of', 'did', 'when', 'as', 'again', 'needn', 'mustn', "you've", 'he', "should've", 'ain', 'shouldn', 'it', "didn't", 'during', 'each', 'more', "hasn't", "won't", 'such', 'further', 'into', 'but', 's', 'a', 'because', "mightn't", 'y', 'themselves', 'own', 'ours', "don't", "mustn't", 'wouldn', 'i', 'or', "haven't", 'haven', 'his', 'only', 'they', 'don', 'him', 'll', 't', "shan't", 'out', 'what', 'which', 'most', 'under', 'an', 'if', 'them', 'then', 'we', 'these', 'any', 'has', 'is', 'nor', 'not', 'will', 'am', 'just', 'some', 'o', 'down', 'than', 'm', 'does', 'no', 'those', "wasn't", 'aren', 'once', 'isn', 'few', 'yours', 'should', 'very', 'ourselves', 'do', 'against', 'that', 'all', 'from', 'over', 'hers', 'for', 'after', 'about', "couldn't", 'she', 'same', "needn't", "wouldn't", 'd', 'the', 'can', 'shan', 'ma', 'its', 'yourselves', 'whom', "you're", 're', 've', 'their', "that'll", 'before', 'other', 'being', "she's", "you'll", 'yourself'

In [5]:
import numpy as np
import nltk
import pandas as pd
import re

from bs4 import BeautifulSoup

def preprocessing(text):
    clean_data = []
    for x in (text[:][0]):
        clean_data =  BeautifulSoup(x, 'html.parser').get_text()
        clean_data = clean_data.lower()#to lower
        clean_data = re.sub(r'[^\w\s]', '', clean_data) # remove punctuation
        #clean_data = nltk.clean_html(clean_data)=>NotImplementedError: To remove HTML markup, use BeautifulSoup's get_text() function
    return clean_data


def preprocessingdf(text):
    #print(text)
    #clean_data = []
    clean_data = text
    clean_data = clean_data.lower()#to lower
    clean_data = BeautifulSoup(clean_data, 'html.parser').get_text()
    return clean_data

test = ['You<p> like &nbsp; to eat apples. He has eaten many apples because he likes eating.']
test_pd = pd.DataFrame(data = test) #makes this into a panda data frame
clean_test = preprocessing(test_pd) #removes punctuation, see above
clean_test

'you like \xa0 to eat apples he has eaten many apples because he likes eating'

In [6]:
data_question['body'][1:9]

1    <p>I accidentally committed the wrong files to...
2    <p>I want to delete a branch both locally and ...
3    <blockquote>\n  <p><strong>Moderator Note:</st...
4    <p>I've been messing around with <a href="http...
5    <p>What is the use of the <code>yield</code> k...
6    <p>After reading <a href="http://groups.google...
7    <p>I mistakenly added files to git using the c...
8    <p>I don't want to rename a remote branch, as ...
Name: body, dtype: object

In [7]:
data_question['body'][1:9].apply(preprocessingdf)
#data_question['clean_body'] = data_question['body'].apply(lambda x : preprocessing(x))


1    i accidentally committed the wrong files to gi...
2    i want to delete a branch both locally and rem...
3    \nmoderator note: given that this question has...
4    i've been messing around with json for some ti...
5    what is the use of the yield keyword in python...
6    after reading hidden features and dark corners...
7    i mistakenly added files to git using the comm...
8    i don't want to rename a remote branch, as des...
Name: body, dtype: object

In [8]:
data_preprocessed = data_question['body'][1:9].apply(tagText.preprocessing)
print(data_preprocessed)

1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3    moderator note given question already sixtysev...
4    ive messing around json time pushing text hurt...
5    use yield keyword python do example im trying ...
6    reading hidden features dark corners cstl comp...
7    mistakenly added files git using command git a...
8    want rename remote branch described rename mas...
Name: body, dtype: object


In [9]:
tagText.cleanIngred('i accidentally ')

'accidentally'

In [10]:
data_question.head()

,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [11]:
data_question.iteritems()

<generator object DataFrame.iteritems at 0x00000232B69631A8>

In [12]:
import nltk
from collections import defaultdict
    
corpora = defaultdict(list)
        
tokenizer = nltk.RegexpTokenizer(r'\w+')
for id, row in data_preprocessed.iteritems():
    for word in tokenizer.tokenize(row):
        if not word in corpora :
            corpora[word] = 0
        #print(word)
        corpora[word] += 1
    

#print(corpora)
print(sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

[('branch', 16), ('git', 13), ('remote', 7), ('distance', 6), ('rename', 5), ('originbugfix', 5), ('maxdist', 5), ('x', 4), ('local', 4), ('remotesoriginbugfix', 3), ('files', 3), ('json', 3), ('delete', 3), ('commit', 3), ('id', 3), ('yield', 3), ('mindist', 3), ('error', 2), ('called', 2), ('want', 2), ('like', 2), ('new', 2), ('returned', 2), ('uptodate', 2), ('question', 2), ('deleted', 2), ('int', 2), ('type', 2), ('remotely', 2), ('selfleftchild', 2), ('locally', 2), ('pushed', 2), ('found', 2), ('result', 2), ('yet', 2), ('0', 2), ('d', 2), ('selfrightchild', 2), ('undo', 2), ('know', 2), ('selfmedian', 2), ('well', 2), ('code', 2), ('one', 2), ('candidates', 2), ('pull', 2), ('already', 2), ('python', 2), ('visual', 1), ('properly', 1), ('following', 1), ('applicationxjavascript', 1), ('contributing', 1), ('api', 1), ('of', 1), ('attempts', 1), ('hidden', 1), ('getchildcandidatesself', 1), ('jochen', 1), ('return', 1), ('resultextendnodevalues', 1), ('gcc', 1), ('browser', 1), 

In [13]:
def count_word_occurencies(df):
        corpora = defaultdict(list)                
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for id, row in df.iteritems():
            for word in tokenizer.tokenize(row):
                if not word in corpora :
                    corpora[word] = 0
                #print(word)
                corpora[word] += 1
        
        corpora = dict((k, v) for k, v in corpora.items() if v > 1)
        return (sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

In [14]:
count_word_occurencies(data_preprocessed)

[('branch', 16),
 ('git', 13),
 ('remote', 7),
 ('distance', 6),
 ('rename', 5),
 ('originbugfix', 5),
 ('maxdist', 5),
 ('local', 4),
 ('x', 4),
 ('delete', 3),
 ('yield', 3),
 ('mindist', 3),
 ('remotesoriginbugfix', 3),
 ('commit', 3),
 ('files', 3),
 ('id', 3),
 ('json', 3),
 ('error', 2),
 ('pull', 2),
 ('know', 2),
 ('selfmedian', 2),
 ('well', 2),
 ('deleted', 2),
 ('yet', 2),
 ('int', 2),
 ('already', 2),
 ('code', 2),
 ('0', 2),
 ('called', 2),
 ('returned', 2),
 ('want', 2),
 ('like', 2),
 ('selfrightchild', 2),
 ('one', 2),
 ('new', 2),
 ('d', 2),
 ('remotely', 2),
 ('selfleftchild', 2),
 ('undo', 2),
 ('candidates', 2),
 ('locally', 2),
 ('uptodate', 2),
 ('python', 2),
 ('found', 2),
 ('result', 2),
 ('type', 2),
 ('question', 2),
 ('pushed', 2)]

In [15]:
print(tagText.count_word_occurencies(data_preprocessed))

[('branch', 16), ('git', 13), ('remote', 7), ('distance', 6), ('rename', 5), ('originbugfix', 5), ('maxdist', 5), ('local', 4), ('x', 4), ('delete', 3), ('yield', 3), ('mindist', 3), ('remotesoriginbugfix', 3), ('commit', 3), ('files', 3), ('id', 3), ('json', 3), ('error', 2), ('pull', 2), ('know', 2), ('selfmedian', 2), ('well', 2), ('deleted', 2), ('yet', 2), ('int', 2), ('already', 2), ('code', 2), ('0', 2), ('called', 2), ('returned', 2), ('want', 2), ('like', 2), ('selfrightchild', 2), ('one', 2), ('new', 2), ('d', 2), ('remotely', 2), ('selfleftchild', 2), ('undo', 2), ('candidates', 2), ('locally', 2), ('uptodate', 2), ('python', 2), ('found', 2), ('result', 2), ('type', 2), ('question', 2), ('pushed', 2)]


In [16]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
word_occurence_count = tagText.count_word_occurencies(data_preprocessed)
word_occurence_count_df = pd.DataFrame.from_dict(word_occurence_count).describe()

In [17]:
word_occurence_count_df.head(100)

,1
count,75991.000000
mean,34.263321
std,315.279064
min,2.000000
25%,2.000000
50%,3.000000
75%,6.000000
max,20052.000000


In [18]:
word_occurence_count_df.plot(kind='bar', color="#f56900", title='occurence of word')

In [19]:
dict_tag = count_word_occurencies(data_preprocessed)
dict_tag = str(dict_tag)
dict_tag

"[('like', 20052), ('using', 18156), ('im', 17409), ('use', 16357), ('code', 15098), ('want', 14713), ('get', 14451), ('way', 14095), ('would', 13548), ('file', 13163), ('new', 12636), ('error', 11625), ('this', 11259), ('one', 11081), ('string', 10799), ('1', 10739), ('class', 10381), ('public', 9653), ('function', 9527), ('return', 9350), ('know', 8714), ('need', 8502), ('ive', 7792), ('0', 7790), ('following', 7762), ('something', 7598), ('example', 7593), ('it', 7412), ('2', 7389), ('set', 7275), ('c', 7081), ('data', 7061), ('work', 6996), ('id', 6952), ('method', 6896), ('value', 6598), ('var', 6571), ('find', 6264), ('name', 6189), ('trying', 6110), ('also', 6085), ('div', 5995), ('int', 5980), ('object', 5880), ('could', 5785), ('tried', 5660), ('question', 5639), ('line', 5498), ('git', 5486), ('time', 5468), ('type', 5426), ('files', 5418), ('make', 5360), ('run', 5257), ('see', 5207), ('problem', 5176), ('list', 5157), ('user', 5032), ('project', 4969), ('possible', 4959), (

In [20]:
tagText.unsupervised_tag(dict_tag, "like git branche totot, test", 2)

['like', 'git']

In [21]:
def unsupervised_tag(dict_word_key, new_question, number_max_tag):
    tags = [word for word in new_question.split() if (word in dict_word_key)]
    return tags[0:number_max_tag]

print(unsupervised_tag(dict_tag, "like git branche totot, test", 20))
print(unsupervised_tag(dict_tag, "like git branche totot, test", 2))
#for word in "like git branche totot, test".split() if (word in dict_word_key)

['like', 'git', 'branche', 'test']
['like', 'git']


In [22]:
for word in "like git branche totot, test".split() :
    print(word)
    if word in str(dict_tag) :
        print("!!!Yes!!!")

like
!!!Yes!!!
git
!!!Yes!!!
branche
!!!Yes!!!
totot,
test
!!!Yes!!!


In [23]:
print(nltk.word_tokenize(data_question['body'][0])[0:10])
print(nltk.tokenize.WhitespaceTokenizer().tokenize(data_question['body'][0])[0:10])
# Tokenize using Punctuations
print(nltk.tokenize.WordPunctTokenizer().tokenize(data_question['body'][0])[0:10])
print(nltk.tokenize.TreebankWordTokenizer().tokenize(data_question['body'][0])[0:10])

['<', 'p', '>', 'Here', 'is', 'a', 'piece', 'of', 'C++', 'code']
['<p>Here', 'is', 'a', 'piece', 'of', 'C++', 'code', 'that', 'shows', 'some']
['<', 'p', '>', 'Here', 'is', 'a', 'piece', 'of', 'C', '++']
['<', 'p', '>', 'Here', 'is', 'a', 'piece', 'of', 'C++', 'code']


In [24]:

from string import punctuation as ponctuation
ponctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'